# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [41]:
import pandas as pd
import numpy as np
import scipy.stats as stats


In [3]:
df = pd.read_csv('homepage_actions.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [5]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


6328

In [29]:
df[['group', 'action']].value_counts()

group       action
control     view      3332
experiment  view      2996
control     click      932
experiment  click      928
dtype: int64

In [35]:
sorted_df = df.sort_values('timestamp', ascending=True).copy()
print(len(sorted_df['id'].unique()))
print(sorted_df.id.duplicated().sum())

6328
1860


In [31]:
nodup_df = sorted_df.drop_duplicates(subset='id', keep='first')
nodup_df[['group', 'action']].value_counts()

group       action
control     view      3332
experiment  view      2996
dtype: int64

In [33]:
filt = sorted_df['id'].duplicated()
only_dup_df = sorted_df.loc[filt]
only_dup_df[['group', 'action']].value_counts()

group       action
control     click     932
experiment  click     928
dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [42]:
control_views = df[['group', 'action']].value_counts()['control']['view']
control_click = df[['group', 'action']].value_counts()['control']['click']

experiment_views = df[['group', 'action']].value_counts()['experiment']['view']
experiment_click = df[['group', 'action']].value_counts()['experiment']['click']

observations = np.array([experiment_click, experiment_views - experiment_click])
expectations = np.array([control_click, control_views - control_click])

stats.chisquare(f_obs=observations, f_exp=expectations)

Power_divergenceResult(statistic=45.943834048640916, pvalue=1.2169234221091101e-11)

In [44]:
experiment_proportion = experiment_click / experiment_views
print(f'Proportion Experiment Converted: {experiment_percent:.5}')
control_proportion = control_click / control_views
print(f'Proportion Control Converted: {control_percent:.5}')

print(f'Difference between experiment & control {experiment_proportion - control_proportion:.3}')

Proportion Experiment Converted: 30.975
Proportion Control Converted: 27.971
Difference between experiment & control 0.03


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [47]:
#Your code here
expected_click = int(experiment_views * control_proportion)
expected_click

838

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [51]:
#Your code here
n = experiment_views
p = control_proportion
stdev_binom = np.sqrt(n*p*(1-p))

num_stdevs = (experiment_click - expected_click) / stdev_binom
num_stdevs

3.6632201602088235

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [53]:
#Your code here
mu = 0
sigma = 1
stats.norm(loc=mu,scale=sigma).sf(num_stdevs)

0.0001245321456477967

In [54]:
#stats.binom(k=k, n=n, p=p).sf()
k = experiment_click
stats.binom.sf(k=k, n=n, p=p)

0.0001323581123494348

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: No. The p-value for the chi-squared test is much more significant than the p-value from the binomial test

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.